# Assignment 1

1. Реализовать Блум фильтр с помощью битового массива.
Например, вы можете использовать [Битовые операции](https://wiki.python.org/moin/BitwiseOperators) или библиотеку bitarray.

2. Провести численный эксперимент при false postive rate = 0.05, и количестве объектов S = 1 000 000.
Убедится, полученные на семинаре оптимальные параметры фильтра позволяют достичь заявленного false positive rate.
Посчитать  $\frac {\epsilon - \hat \epsilon} \epsilon$, где $\hat \epsilon$ - ваша экспериментальная оценка false positive rate. В качестве объектов используйте строки.

## Фильтр Блума

In [1]:
import random
import math
from bitarray import bitarray 
import string

In [31]:
class BloomFilter(object):
    
    def __init__(self, false_positive, S):
        self.a_size = math.ceil((S * math.log(1/false_positive))/(math.log(2)**2))
        self.k_func = math.ceil((self.a_size/S) * math.log(2))
        self.array = bitarray(self.a_size)
        self.array.setall(0)
        self.primes = [random.choice([1301017, 4271387, 1597979, 1885043, 5064701, 
                                      2469157, 7384033, 8976301, 7554487, 9925439]) for _ in range(self.k_func)]
        self.xs = [random.randint(1, self.primes[i]) for i in range (self.k_func)]
        
    def add(self, value):
        for x in self.multi_hash(value):
            self.array[x] = True
    
    def check(self, value):
        for x in self.multi_hash(value):
            if self.array[x] == False: return False
        return True
    
    def _hash_str(self, value, p, x):
        h = 0
        for i, s in enumerate(value):
            h += ord(s) * x**i % p
        return h % p
                
    def multi_hash(self, value):
        for x in range(self.k_func):
            h = self._hash_str(value, self.primes[x], self.xs[x])
            yield h % self.a_size

## Test

In [3]:
''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,50)))

'J8lc2PT29xkGSMz91FddrCsZm'

In [4]:
def test_bloomfilter(p, n, w = 100):
    bf = BloomFilter(p, n)
    train = set(''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,w))) for _ in range(n))
    #train = set(''.join(random.choices(string.ascii_lowercase, k=random.randint(1,w))) for _ in range(n))
    for i in train:
        bf.add(i)
    TP, FP, TN, FN = 0, 0, 0, 0
    for _ in range(n):
        value = ''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,w)))
        res = bf.check(value)
        if res is True:
            if value in train: TP += 1
            else: FP += 1
        else:
            if value in train: FN += 1
            else: TN += 1
    TP = TP / n
    FP = FP / n
    TN = TN / n
    FN = FN / n
    #print(bf.primes, bf.xs)
    #print ('Size={}, k={}, TP={}, FP={}, TN={}, FN={}, BA={}'.format(bf.a_size, bf.k_func, TP, FP, TN, FN, sum(bf.array)/len(bf.array)))
    return FP

In [36]:
test_bloomfilter(0.05, 10**4)

0.0508

In [21]:
test_bloomfilter(0.05, 10**5)

0.04962

In [ ]:
array = []
for _ in range(5):
    result = test_bloomfilter(0.05, 10**6)
    print (result)
    array.append(result)

In [34]:
experimental = sum(array)/5

In [35]:
(0.05 - experimental)/0.05

-0.02799999999999997